In [19]:
import pandas as pd
from rdkit import Chem
import os
import numpy as np

from tqdm import tqdm
import MDAnalysis as mda

In [2]:
af_human_proteome = os.listdir('/Users/kamen/Downloads/UP000005640_9606_HUMAN_v4/unzipped/')

In [3]:
hct116 = pd.read_csv('../hct116_without_CR.csv')

In [4]:
unique_proteins = set()
dupes = [x for x in af_human_proteome if x.split('-')[1] in unique_proteins or unique_proteins.add(x.split('-')[1])]

In [6]:
len(set(hct116['uid']))

6089

In [5]:
len(unique_proteins.intersection(set(hct116['uid'])))

5988

There are 101 proteins in the assay that are not in the AF predictions

In [7]:
hct116['in_af'] = hct116['uid'].apply(lambda x: x in unique_proteins)

These UniProt IDs are not in the AF set - I guess it is because they are from another organism (tr not sp)

In [8]:
hct116.loc[~hct116['in_af']]

,Uniprot ID,Site Position,Gene Symbol,uid,in_af
83,tr|F5GX68|F5GX68_HUMAN,1476,DNMT1,F5GX68,False
84,tr|F5GX68|F5GX68_HUMAN,1478,DNMT1,F5GX68,False
210,tr|B3V0L1|B3V0L1_HUMAN,36,ARL6IP4,B3V0L1,False
211,tr|B3V0L1|B3V0L1_HUMAN,48,ARL6IP4,B3V0L1,False
237,sp|P36969|GPX4_HUMAN,93,GPX4,P36969,False
...,...,...,...,...,...
22988,tr|E9PHB3|E9PHB3_HUMAN,234,ZNF384,E9PHB3,False
22989,tr|E9PHB3|E9PHB3_HUMAN,261,ZNF384,E9PHB3,False
22990,tr|E9PHB3|E9PHB3_HUMAN,264,ZNF384,E9PHB3,False
23054,tr|J3KR51|J3KR51_HUMAN,357,ZNF525,J3KR51,False


In [9]:
hct116 = hct116.loc[hct116['in_af']]
hct116 = hct116.drop(columns='in_af')

Find all the Cys S atoms

In [10]:
chain_id = []
sulfur_name = []
cys_idx = []
pdb_ids = [] 
file_names = []
visited = set()
for pdb in tqdm(af_human_proteome):
    if (pdb.split('-')[1]) not in visited:
        visited.add(pdb.split('-')[1])
        with open(os.path.join('/Users/kamen/Downloads/UP000005640_9606_HUMAN_v4/unzipped', pdb), 'r') as f:
            for line in f.readlines():
                line_li = line.split()
                if len(line_li) == 12 and line_li[2].startswith('S') and line_li[3] == 'CYS':
                    pdb_ids.append(pdb.split('-')[1])
                    cys_idx.append(line_li[5])
                    sulfur_name.append(line_li[2])
                    chain_id.append(line_li[4])
                    file_names.append(pdb)
        f.close()

100%|██████████| 23392/23392 [01:31<00:00, 254.83it/s]


In [13]:
af_human_proteome_df = pd.DataFrame(columns=['uid', 'cys_idx', 'sulfur_name','chain_id', 'file_name'])
af_human_proteome_df['uid'] = pdb_ids
af_human_proteome_df['cys_idx'] = cys_idx
af_human_proteome_df['sulfur_name'] = sulfur_name
af_human_proteome_df['chain_id'] = chain_id
af_human_proteome_df['file_name'] = file_names

In [11]:
hct116['Site Position'] = hct116['Site Position'].apply(lambda x: str(x))
hct116['uid'] = hct116['uid'].apply(lambda x: x.split('-')[0])
hct116['combined_id_idx'] = hct116[['uid', 'Site Position']].apply(tuple, axis=1)
positive_tuples = set(hct116['combined_id_idx'].values)

These are all the cysteines that Kuljanin saw (the ones that bound a the proe in their study)

In [12]:
len(positive_tuples)

23124

In [15]:
af_human_proteome_df['combined_id_idx'] = af_human_proteome_df[['uid', 'cys_idx']].apply(tuple, axis=1)

In [16]:
af_human_proteome_df['positive'] = af_human_proteome_df['combined_id_idx'].apply(lambda x: x in positive_tuples)

In [18]:
af_human_proteome_df['positive'].value_counts()

False    209913
True      18361
Name: positive, dtype: int64

Nearly only 10% of the cysteines are ligandable - can slightly increase this number if we filter out the disulfide bridges

In [20]:
for i,row in tqdm(af_human_proteome_df.iterrows()):
    uid,cys_idx, pdb = row[0],row[1], row[-3]
    prot = mda.Universe(os.path.join('/Users/kamen/Downloads/UP000005640_9606_HUMAN_v4/unzipped/',pdb))
    prot = prot.select_atoms("protein")
    selected_atoms = prot.select_atoms(f"byres sphzone 10.0 name SG and resid {cys_idx} and segid A")
    selected_atoms.write(f'cysteine_environments_10a_pdb/{uid}_{cys_idx}.pdb')

0it [00:00, ?it/s]/Users/kamen/opt/anaconda3/envs/DeepCoSI/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/Users/kamen/opt/anaconda3/envs/DeepCoSI/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
228274it [6:40:15,  9.51it/s] 


Du et al. have code to detect the disulfide bridges directly from the pdb file, but in my case these are not indecated in the pdb file

In [22]:
af_human_proteome_df = af_human_proteome_df.sample(frac=1).reset_index()

In [28]:
af_human_proteome_df = af_human_proteome_df.drop('index', axis=1)

Randomize the protein in the dataset so we don't introduce any bias there

In [29]:
af_human_proteome_df.to_csv('af_human_proteome.csv', index=False)

For the new notebook

In [ ]:
af_human_proteome_df = pd.read_csv('af_human_proteome.csv')

In [ ]:
keys = [f'{row[0]}_{row[1]}' for i,row in af_human_proteome_df.iterrows()]
labels = af_human_proteome_df['positive'].values
cys_chain_and_idx = [(row[-2], int(row[1])) for i,row in af_human_proteome_df.iterrows()]
pdb_pockets = ['cysteine_environments_10a_pdb/' + key + '.pdb' for key in keys]

In [ ]:
from DeepCoSI_GraphGenerate import graphs_from_pdb_pocket, collate_fn, GraphDatasetGenerateSingleThread

In [ ]:
for pdb_pocket,key,label,cys in tqdm(zip(pdb_pockets, keys,labels,cys_chain_and_idx)):
    graphs = graphs_from_mol(pdb_pockets, key, label, cys, graph_dic_path='cysteine_pockets_as_rdkit_mols', path_marker='/', EtaR=4.00, ShfR=0.5, Zeta=8.00, ShtZ=0)

In [ ]:
train_dataset = GraphDatasetGenerateSingleThread(keys=keys[:150000], labels=labels[:150000],cyss=cys_chain_and_idx[:150000],
                                       graph_ls_path='cached_datasets',
                                       graph_dic_path='cysteine_pockets_as_rdkit_mols',
                                       name_of_data='training',
                                       path_marker='/')

In [ ]:
validation_dataset = 